In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. In this notebook, we use ChatGPT, Gemini on GenerativeAI and Anthropic Claude.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message class allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import InMemoryConversationMemory, JSONConversationMemory
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
from langrila import ToolConfig, ToolParameter, ToolProperty

tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### ChatGPT

ToolConfig object is internally converted to client tool configs

In [7]:
chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chatgpt.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect ambiance. Let the fun begin! 🎉🕺💃')], name=None), usage=Usage(prompt_tokens=132, completion_tokens=47, total_tokens=179), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_3LJbJLjLy8XiZ6iKjxpc2m3J', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_JH35dBqUNcaA5QIH0muf9t6T', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_ESZnFoCzYsHRVQwrKdRSXICx', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': 'tool'

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect ambiance. Let the fun begin! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_3LJbJLjLy8XiZ6iKjxpc2m3J',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_JH35dBqUNcaA5QIH0muf9t6T',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_ESZnFoCzYsHRVQwrKdRSXICx',
     'function': {'arguments': '{"b

Asynchronous generation

In [10]:
response = await chatgpt.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is set! 🎉\n\n- The energetic music is playing at a lively 120 BPM.\n- The lights have been dimmed for a cozy atmosphere.\n- The disco ball is spinning, adding that perfect touch of fun.\n\nGet ready to dance and enjoy the night! 🕺💃')], name=None), usage=Usage(prompt_tokens=133, completion_tokens=62, total_tokens=195), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_GUJvXF3OsJDvd4hG0jO1DyUn', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_oe5vo4HhhsjKChKlyKGpYW6j', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}, {'id': 'call_ZlReX5mdEYtoIcEFtjz78Jsl', 'function': {'arguments': '{"power": true}', 

multiple generation

In [11]:
response = chatgpt.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is pumping at 120 BPM, and the lights are dimmed to create a cozy atmosphere. Let the festivities begin! 🎉💃🕺'},
  {'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect atmosphere. Let the fun begin! 🎉💃🕺'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chatgpt.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The music is live! Now, let’s set the atmosphere:\n\n1. **Lighting:** Dim the lights and add some colorful LED lights or fairy lights to create a cozy vibe.\n  \n2. **Decorations:** Use balloons, streamers, or themed decorations to liven up the space.\n\n3. **Seating Area:** Arrange comfortable seating with cushions and blankets where guests can lounge and chat.\n\n4. **Food & Drinks:** Set up a snack and drink station with appetizers, finger foods, and a signature cocktail or mocktail for everyone to enjoy.\n\n5. **Games & Activities:** Prepare some fun party games or activities to keep the energy high and guests engaged.\n\n6. **Chill Zone:** Create a quieter area with softer music for guests who want to take a break from the dance floor.\n\nWith the energetic music pumping, everyone will be in the party mood in no time! Enjoy! 🎉'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 62,
  '

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chatgpt.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=169, completion_tokens=75, total_tokens=244), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_UxiyKuqXUw6QLwtesPh5XgHB', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_7I2NhUPHCsxtg2Dw4pbildoI', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_9ShtcWt0BqnimqVX8S0IKK2N', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_UxiyKuqXUw6QLwtesPh5XgHB'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_7

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 169,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_UxiyKuqXUw6QLwtesPh5XgHB',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_7I2NhUPHCsxtg2Dw4pbildoI',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_9ShtcWt0BqnimqVX8S0IKK2N',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_UxiyKuqXUw6QLwtesPh5XgHB'}

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_UxiyKuqXUw6QLwtesPh5XgHB', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_7I2NhUPHCsxtg2Dw4pbildoI', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_9ShtcWt0BqnimqVX8S0IKK2N', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_UxiyKuqXUw6QLwtesPh5XgHB'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_7I2NhUPHCsxtg2Dw4pbildoI'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_9ShtcWt0BqnimqVX8S0IKK2N')], name=None)

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for ChatGPT.

In [17]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [18]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, some energetic music playing, and I've dimmed the lights.  Anything else I can do to set the mood? Snacks? Drinks? Games? 😉 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 43},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
        

Asynchronous generation

In [19]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 29},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      

Gemini allows you to call specific tool as well as ChatGPT.

In [20]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  🎶 \n\nLet's get this party started! 🎉  What else can I do to make it awesome?  Need some lighting suggestions? 😉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 68,
  'completion_tokens': 40},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
           

### Gemini on VertexAI

In [21]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [22]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm getting the party started! \n\nI've queued up some upbeat music with a tempo of 120 beats per minute. It's not too loud, just enough to create a fun atmosphere. \n\nI've also dimmed the lights to a comfortable level, creating a warm and inviting ambiance.\n\nAnd finally, I've turned on the disco ball!  It's spinning and reflecting colorful lights, adding a touch of excitement and fun.\n\nLet's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 73,
  'completion_tokens': 111},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
    

multiple generation

In [23]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Alright, party time! 🎶 I've cranked up some upbeat music with a good beat (around 120 BPM), dimmed the lights to a cozy level, and fired up the disco ball!  ✨  \n\nLet's get this party started! 🎉 \n"},
  {'text': "Alright, party time! 🎶 I've cranked up some upbeat music with a good beat (around 120 BPM), dimmed the lights to a cozy level, and fired up the disco ball!  ✨  \n\nLet's get this party started! 🎉 \n"}],
 'name': None}

### Anthropic Claude

tool_configs can be shared with Claude.

In [24]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [25]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 60% brightness (0.6). This creates a cozy atmosphere without making it too dark.\n2. Started some music that is:\n   - Energetic (to keep the party vibe)\n   - Not too loud (for comfort)\n   - At a moderate tempo of 120 BPM (good for casual dancing and conversation)\n3. Turned on the disco ball for some fun, dynamic lighting effects.\n\nThis setup should create a lively yet comfortable party atmosphere. The dimmed lights and spinning disco ball will create an interesting visual environment, while the energetic but not overly loud music at a moderate tempo should be perfect for both chatting and dancing.\n\nIs there anything you'd like to adjust about this setup? For example, if you want the lights brighter or dimmer, the music louder or quieter, or if you'd like to turn off the disco ball, just let me know and I can make those 

Asynchronous generation

In [26]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Turned on the disco ball to add some fun, dynamic lighting to the space.\n2. Started some music that's energetic but not too loud, with a moderate tempo of 120 BPM. This should create a lively atmosphere without being overwhelming.\n3. Dimmed the lights to 60% brightness, which should provide a cozy, party-like ambiance without being too dark.\n\nThese settings should create a comfortable party mood that's lively and fun, but not too intense. The spinning disco ball will add a festive touch, the music will keep the energy up without being too loud, and the dimmed lights will create a relaxed atmosphere.\n\nIs there anything else you'd like me to adjust to make the party mood even more comfortable for you?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 939,
  'completion_tokens': 191},
 'prompt': [{'ro

### Claude on Amazon Bedrock

In [27]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [28]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've made the following adjustments to create a comfortable party mood:\n\n1. Dimmed the lights: I've set the brightness to 0.4 (on a scale of 0.0 to 1.0), which should provide a warm, inviting glow without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party atmosphere, but set it to not be too loud so people can still comfortably chat. The beats per minute (bpm) is set to 120, which is a good middle ground for a variety of party music styles.\n\n3. Powered on the disco ball: This will add a fun, party-like ambiance to the room with moving lights.\n\nThese settings should create a comfortable party mood that's lively enough for fun but not overwhelming. The dimmed lights and spinning disco ball will create a nice visual atmosphere, while the energetic but not-too-loud music will provide a good backdrop for socializing and dancing.\n\nIs there anything else you'd like to adjust? For examp

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [29]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power']), type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm']), type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lights.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToo

Then these client tool config objects is transformed to client tools by format() method.

In [30]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}},
   'required': ['power']}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [31]:
from langrila.gemini.genai import GeminiToolConfig

In [32]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power'])),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm'])),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='brightness', type=

In [33]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For ChatGPT

In [34]:
chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [35]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The atmosphere is now perfect for a party! The lights are dimmed to a cozy level, the disco ball is spinning beautifully, and energetic music with a lively beat is playing. Get ready to dance and have a great time! 🎉✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 133,
  'completion_tokens': 49},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Rdbng8ijTt5jF3BHDeTD9msY',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_XCRYAXUQOKFDTt8KIJvEW2UU',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_tYSPq42NjQ5IEZpUWcM5HWoq',
     'function': {'arguments': '

Next turn

In [36]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music's BPM has been increased to a high-energy 180! Get ready to dance even harder! 🎶💃🕺"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 243,
  'completion_tokens': 28},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Rdbng8ijTt5jF3BHDeTD9msY',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_XCRYAXUQOKFDTt8KIJvEW2UU',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_tYSPq42NjQ5IEZpUWcM5HWoq',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool'

### For Gemini

In [37]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [38]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright! Disco ball's spinning, some upbeat music is playing, and the lights are dimmed.  What else can I do to set the mood?  Need some snacks, drinks, or maybe some games? 🎉 🎶 😄 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 47},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value

In [39]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current music is playing at **120 bpm**.  🎧 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 202,
  'completion_tokens': 14},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "dim_lights"
  

In [40]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've cranked things up!  The music is now bumping at a solid 180 bpm.  Let's get this party moving! 💃🕺 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 285,
  'completion_tokens': 37},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
   

## Multi-turn conversation using tools with multiple model

In [41]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [42]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt)

In [43]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create a cozy atmosphere. Let the festivities begin! 🎉🌟'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 44},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_EGOA50CfuLY2WjUaGmfPm3Ae',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_egauNEsHjMRLmgAAq3h4tRMW',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7gv6JRiSMNgB8ghjjLnFFEtN',
     'function': {'arguments': 

Gemini can continue conversation without any additional process.

In [44]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music is pumping now! I've cranked it up to a heart-pounding 180 BPM. Get ready to move! 🎶🔥 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 257,
  'completion_tokens': 33},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {

Gemini also works as well.

NOTE : Claude may not work in multi-turn conversation using tools. Bug fix is in progress.

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [45]:
from langrila import InMemoryConversationMemory

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [46]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt, tool_choice="auto")

In [47]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [48]:
print(response.message.content[0].text)

The music BPM has been increased to 180! Let's pump up the energy and keep the party going! 🎶💥


In [49]:
response.usage

Usage(prompt_tokens=209, completion_tokens=25, total_tokens=234)

In [50]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_yWbs0vyyIUs3UWgjKSBk6ZKS',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_tdd7A531OWXyEg6kCgFPVHDL',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_RDK53q5VKOQTODF5vvpY12is',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_yWbs0vyyIUs3UWgjKSBk6ZKS',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_tdd7A531OWXyEg6kCgFPVHDL',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm='120'"},
 {'role': 'tool',
  'tool_call_id': 'call_RDK53q5VKOQTODF5vvpY12is',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 

In [51]:
# whole history
chatgpt.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_yWbs0vyyIUs3UWgjKSBk6ZKS'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_tdd7A531OWXyEg6kCgFPVHDL'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_RDK53q5VKOQTODF5vvpY12is'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_yWbs0vyyIUs3UWgjKSBk6ZKS',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_tdd7A531OWXyEg6kCgFPVHDL',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'}],
  'name': None},
 {